In [1]:
import sys

sys.path.append("../..")
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from transformers import AutoTokenizer, LlamaForCausalLM

from src.hyperdas.data_utils import (
    filter_dataset,
    generate_ravel_dataset,
)

In [3]:
model_name_or_path = "meta-llama/Meta-Llama-3-8B"

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
model = LlamaForCausalLM.from_pretrained(
    model_name_or_path, torch_dtype=torch.bfloat16, device_map={"": 0}
)

tokenizer.padding_side = "left"
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

In [ ]:
all_attributes = [
    "Country",
    "Continent",
    "Language",
    "Latitude",
    "Longitude",
    "Timezone",
]

for split in ["train", "test"]:
    dataset = generate_ravel_dataset(
        10000,
        root_path="/workspace/HyperDAS/assets/data/ravel",
        target_attributes=["Country", "Continent"],
        isolate_attributes=list(set(all_attributes) - set(["Country", "Continent"])),
        template_split=split,
        entity_split=split,
    )

    dataset = filter_dataset(model, tokenizer, dataset, batch_size=16)
    dataset = filter_dataset(model, tokenizer, dataset, batch_size=16)
    dataset = filter_dataset(model, tokenizer, dataset, batch_size=16)

    dataset.save_to_disk(
        f"/workspace/HyperDAS/experiments/RAVEL/data/city_country_{split}"
    )

In [ ]:
# CHANGE: {base_entity} → {source_entity} | ATTR: {target_attribute}
all_attributes = [
    "Country",
    "Continent",
    "Language",
    "Latitude",
    "Longitude",
    "Timezone",
]

for split in ["train", "test"]:
    dataset = generate_ravel_dataset(
        10000,
        root_path="/workspace/HyperDAS/assets/data/ravel",
        target_attributes=["Country", "Continent"],
        isolate_attributes=list(set(all_attributes) - set(["Country", "Continent"])),
        template_split=split,
        entity_split=split,
        edit_instruction_template="CHANGE: {base_entity} -> {source_entity} | ATTR: {random_target_attribute}",
    )

    dataset = filter_dataset(model, tokenizer, dataset, batch_size=16)
    dataset = filter_dataset(model, tokenizer, dataset, batch_size=16)
    dataset = filter_dataset(model, tokenizer, dataset, batch_size=16)

    dataset.save_to_disk(
        f"/workspace/HyperDAS/experiments/RAVEL/data/city_country_{split}_v0.1"
    )

In [ ]:
all_attributes = [
    "Country",
    "Continent",
    "Language",
    "Latitude",
    "Longitude",
    "Timezone",
]

for attribute in all_attributes:
    for split, size in [("train", 20000), ("test", 4000)]:
        print(f"Generating data for {attribute} in split {split}")
        all_other_attributes = [a for a in all_attributes if a != attribute]

        dataset = generate_ravel_dataset(
            size,
            root_path="/home/ubuntu/HyperDAS/data/ravel",
            target_attributes=[attribute],
            isolate_attributes=all_other_attributes,
            template_split=split,
            entity_split="both",
        )

        dataset = filter_dataset(model, tokenizer, dataset, batch_size=16)
        dataset = filter_dataset(model, tokenizer, dataset, batch_size=16)
        dataset = filter_dataset(model, tokenizer, dataset, batch_size=16)

        dataset.save_to_disk(
            f"/home/ubuntu/HyperDAS/experiments/ravel/data/city_{attribute.lower()}_{split}"
        )

In [13]:
def generate_ravel_causal_dataset(split):
    dataset = generate_ravel_dataset(
        10000,
        root_path="/workspace/HyperDAS/assets/data/ravel",
        target_attributes=["Country"],
        isolate_attributes=[],
        template_split=split,
        entity_split=split,
    )

    dataset = filter_dataset(model, tokenizer, dataset, batch_size=16)
    dataset = filter_dataset(model, tokenizer, dataset, batch_size=16)
    dataset = filter_dataset(model, tokenizer, dataset, batch_size=16)

    dataset.save_to_disk(
        f"/workspace/HyperDAS/experiments/RAVEL/data/ravel_country_causal_only_{split}"
    )

    return dataset

In [ ]:
train_dataset, test_dataset = (
    generate_ravel_causal_dataset("train"),
    generate_ravel_causal_dataset("test"),
)

In [ ]:
train_dataset[0]

In [ ]:
all_country = [d for d in test_dataset if d["attribute"] == "Country"]
all_country_with_isolation = [
    d
    for d in test_dataset
    if d["attribute"] == "Country" and d["attribute_type"] == "isolate"
]
len(test_dataset), len(all_country), len(all_country_with_isolation)

In [ ]:
attr_dict = {}

for d in test_dataset:
    if d["attribute"] not in attr_dict:
        attr_dict[d["attribute"]] = 0

    attr_dict[d["attribute"]] += 1

attr_dict